In [2]:
import sys 

from vmtk import pypes
from vmtk import vmtkscripts

In [7]:
# this function is useful just to input windows path with \ and output the same path but with /  
import pathlib

def goodpath(path):
    p1 = pathlib.PureWindowsPath(path) 
    mast_path = p1.as_posix()
    return(mast_path)

**From original code**

```
mast_dir = ' D:/Hartmann/PythonAll/seal_whiskers/Har20191023_harbor3_jig1/' 

dir_1 = mast_dir + 'saved/IMG-0002-00001.dcm ' ###raw dcm stack

dir_2 = mast_dir + 'image_volume_voi.vti ' ###image: one whisker

dir_5 = mast_dir + 'image_volume_voi_v2.vti ' ###image: many whisker

dir_3 = mast_dir + 'mc_surface.vtp ' ### surface

dir_4 = mast_dir + 'level_sets.vti ' ###image 

arg_1 = 'vmtkimagereader -ifile' + dir_1 + '--pipe vmtkimagevoiselector -ofile' +  dir_5

arg_3 = 'vmtklevelsetsegmentation -ifile' + dir_5 + '-ofile' + dir_4

arg_2 = 'vmtkmarchingcubes -ifile' +  dir_5 +'-l 5.0 -ofile' +  dir_3 + '--pipe vmtksurfaceviewer'

myArguments = 'vmtkmarchingcubes -ifile ' + dir_2 + ' -l 5 --pipe vmtksurfaceviewer'



#myPype = pypes.PypeRun(myArguments)
```

### Directory allocation


In [ ]:
mast_dir = ' C:/Users/corma/Desktop/HLAB_seals/whiskers_temp/d_Har20190708_harbor3_set3_part2/'
dir_1 = mast_dir + 'Har20190708_harbor3_set3_part2_dcm/IMG-0005-00001.dcm ' ###raw dcm stack

# "C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\d_Har20190708_harbor3_set3_part2\Har20190708_harbor3_set3_part2_dcm\IMG-0005-00001.dcm"

dir_2 = mast_dir + 'image_volume_voi.vti ' ###image: one whisker
dir_5 = mast_dir + 'image_volume.vti ' ###image: many whisker
dir_3 = mast_dir + 'mc_surface.vtp ' ### surface
dir_3_less = mast_dir + 'mc_surf_less.vtp '
dir_4 = mast_dir + 'level_sets.vti ' ###image 
dir_6 = mast_dir + 'mc_surf_smoothed.vtp ' ###smoothed surface
dir_6_extra = mast_dir + 'mc_surf_super_smoothed.vtp ' ###extra smoothed surface
dir_7 = mast_dir + 'sb_surf.vtp ' ###surface subdivision
dir_8 = mast_dir + 'mc_surf_clip.vtp ' ###surface with endcaps clipped off
dir_9 = mast_dir + 'smoothed_surf_centerlines.vtp ' ###centerlines of smoothed surface
dir_10 = mast_dir + 'mc_surf_out.stl ' ###output surface for Geomagic
dir_11 = mast_dir + 'isol_surf.vtp ' ###input surface from Geomagic/SW


### Actions pipelines- initial CT scan processing and export to Geomagic

In [ ]:
arg_1 = 'vmtkimagereader -ifile' + dir_1 + '--pipe vmtkimagevoiselector -ofile' +  dir_5

arg_2 = 'vmtkmarchingcubes -ifile' +  dir_5 +'-l -250.0 -ofile' +  dir_3 + '--pipe vmtksurfaceviewer' ###default 5.0, -500 for whiskers, originally arg_2
arg_2_less =  'vmtkmarchingcubes -ifile' +  dir_5 +'-l -250.0 -ofile' +  dir_3_less + '--pipe vmtksurfaceviewer' ###less noise, use for centerlines calcs
arg_3 = 'vmtksurfacesmoothing -ifile' + dir_3_less + '-method "laplace" -relaxation 0.3 -iterations 50 -ofile' + dir_6_extra ###laplace smoothing method, relaxation .07 is good
###arg_3 = 'vmtksurfacesmoothing -ifile' + dir_3 + '-passband 0.1 -iterations 250 -ofile' + dir_6_extra ###taubin smoothing, originally arg_4
arg_4 =  'vmtksurfacereader -ifile' + dir_3_less  + '--pipe vmtksurfacesmoothing -method "laplace" -relaxation 0.3  -iterations 50  --pipe vmtkrenderer --pipe vmtksurfaceviewer -display 0 --pipe vmtksurfaceviewer -i @vmtksurfacereader.o -color 1 0 0 -display 1' ###using laplace smoothing
###arg_4 =  'vmtksurfacereader -ifile' + dir_3  + '--pipe vmtksurfacesmoothing -iterations 250 -passband 0.1 --pipe vmtkrenderer --pipe vmtksurfaceviewer -display 0 --pipe vmtksurfaceviewer -i @vmtksurfacereader.o -color 1 0 0 -display 1' ###taubin smoothing, originally arg_5
arg_5_opt1 = 'vmtksurfacesubdivision -ifile' + dir_6_extra + '-ofile' + dir_7 + '-method butterfly'###increases number of triangles on surface, if used then change dir_6 in arg_6 to dir_7
arg_5_opt2 = 'vmtksurfaceclipper -ifile' + dir_6_extra + '-ofile' + dir_8 
arg_6 = 'vmtkcenterlines -seedselector openprofiles -ifile' + dir_8 + '-ofile' + dir_9

arg_prepping = 'vmtksurfaceclipper -ifile' + dir_3 + '--pipe vmtksurfacewriter -ofile' + dir_10 
arg_7 = 'vmtksurfacewriter -ifile' + dir_3 + '-ofile' + dir_10
arg_8 = 'vmtksurfacereader -ifile' + dir_10 + ' -ofile' + dir_11

arg_help = 'vmtkcenterlines --help' ###replace 'vmtkcenterlines' with any function you want to know about, faster than googling
#vmtksurfaceconnectivity -ifile airway.vtp --pipe vmtksurfacesmoothing -passband 0.01 -iterations 30 --pipe vmtkcenterlines -ofile airway_centerlines.vtp
    

 
arg_1_opt = 'vmtklevelsetsegmentation -ifile' + dir_5 + '-ofile' + dir_4 ###originally arg_3
arg_2_opt = 'vmtkmarchingcubes -ifile ' + dir_2 + ' -l 5 --pipe vmtksurfaceviewer'###vestigial part of original code, 'myArguments'

#run args1-5 sequentially to automatically segment and extract centerlines
#run arg_1_opt for colliding fronts initialization
#myPype = pypes.PypeRun(myArguments)

#### Runs the commands as written above

In [ ]:
myPype = pypes.PypeRun(arg_2)

In [ ]:
myPype = pypes.PypeRun(arg_help) ## for documentation/syntax issues

## Loops through whisker directories and segments them, outputting STL (run functions first)

In [ ]:
# reads in dicom set into vmtk, only needs to be done once for any image set
def arg1creator(mast_path, dicom_path):
    mast_dir = mast_path + "/"
    dir_1 = mast_dir + dicom_path + ' '
    dir_5 = mast_dir + 'image_volume.vti '
    arg_1 = 'vmtkimagereader -ifile ' + dir_1 + ' --pipe vmtkimagewriter -ofile ' + dir_5
    return(arg_1)



In [ ]:
# segments the whiskers. the -500 is the parameter that controls what gets counted as whisker 
 # and what gets counted as air. going lower (e.g. -600) includes more material as whisker, going higher includes less whisker
    
def arg2creator(mast_path):
    mast_dir = mast_path + "/"
    dir_3 = mast_dir + 'mc_surface.vtp '
    dir_5 = mast_dir + 'image_volume.vti '
    arg_2 = 'vmtkmarchingcubes -ifile ' +  dir_5 +' -l -500.0 -ofile ' +  dir_3
    return(arg_2)
    

In [ ]:
# alt functions used initially for higher res scans. alt arg2 also just segments
ddef altarg2creator(mast_path):
    mast_dir = mast_path + "/"
    dir_3 = mast_dir + 'mc_surface.vtp '
    dir_5 = mast_dir + 'image_volume.vti '
    arg_2 = 'vmtkmarchingcubes -ifile ' +  dir_5 +' -l -350.0 -ofile ' +  dir_3
    return(arg_2)

In [ ]:
#outputs STL file
def arg7creator(mast_path):
    mast_dir = mast_path + "/"
    dir_3 = mast_dir + 'mc_surface.vtp '
    dir_10 = mast_dir + 'mc_surf_out.stl ' 
    arg_7 = 'vmtksurfacewriter -ifile ' + dir_3 + '-ofile ' + dir_10
    return(arg_7)

def altarg7creator(mast_path):
    mast_dir = mast_path + "/"
    dir_3 = mast_dir + 'mc_surface.vtp '
    dir_10 = mast_dir + 'mc_surf_out_2.stl ' # the 2 is to keep track of which settings are used for which STL
    arg_7 = 'vmtksurfacewriter -ifile ' + dir_3 + '-ofile ' + dir_10
    return(arg_7)
    

In [ ]:
# this functionizes all the steps necessary to segment a whisker, from importing the dicom set to outputting STL. 
# "root" is the parent folder, "dir" is the folder with dicoms in it

import os
import pathlib
import time
import gc

gc.enable()

def segmenter(root, dir, mode):
    firstdcm = os.listdir(os.path.join(root, dir))[0]

    p1 = pathlib.PureWindowsPath(root) #for mast dir
    mast_path = p1.as_posix()
#             print(mast_path, '\n')
    p2 = pathlib.PureWindowsPath(os.path.join(dir, firstdcm)) #dir 1
    dicom_path = p2.as_posix()

    arg_1 = arg1creator(mast_path, dicom_path)
    
    if mode == 'lowres':
        arg_2 = arg2creator(mast_path)
    elif mode == 'highres':
        arg_2 = alttarg2creator(mast_path)
        
    if mode == 'lowres':    
        arg_7 = arg7creator(mast_path)
    elif mode == 'highres':
        arg_7 = altarg7creator(mast_path)
    
    try:
        FirstPype = pypes.PypeRun(arg_1)
        time.sleep(5)
    except Exception:
        pass

    try:
        SecondPype = pypes.PypeRun(arg_2)
        time.sleep(10)
    except Exception:
        pass

    try:
        ThirdPype = pypes.PypeRun(arg_7)
        time.sleep(10)
    except Exception:
        pass
    gc.collect() #collects garbage after each whisker run, which it will delete a couple whiskers later

In [ ]:
# goes through all the folders in a parent folder and runs the segmenter function on them if there isn't already 
    # an STL output there
import os
import pathlib
import time
import gc

gc.enable() #enables garbage collection, which frees up memory as it goes along 

rootdir = "D:\Hartmann_seal_whiskers\All_whiskers"

for root, dirs, files in os.walk(rootdir):
    for dir in dirs:
        if (dir.endswith("_dcm") == True) and (os.path.exists(os.path.join(root, "mc_surf_out.stl")) == False
            segmenter(root, dir, 'lowres')


## Centerlines for isolated whiskers 

Work in progress, works to output .dat file with coordinates for single whisker centerline, yet to be functionized

In [3]:
import os
import pathlib
import numpy as np

# wpath = r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\d_Har20190708_harbor3_set3_fullLength\whiskers\W2.stl"
wpath  = r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\cur\d_Har20190702_short_whiskers05\whiskers\smoothed\W11_HR_442_t_0001_150.stl"
p1 = pathlib.PureWindowsPath(wpath)
whiskerpath = p1.as_posix()
(pp, whiskerstl) = os.path.split(whiskerpath)
basename = os.path.splitext(whiskerstl)[0]


vobpath = pp + "/" + basename + '.vtp '

clpath = pp + "/" + basename + '_cl.vtp '

cldatpath = pp + "/" + basename + '_cl.dat '

cl_opath = pp + "/" + basename + '_cl.stl '

clcoords = pp + "/" + basename + '_npar.dat'

imin = 'vmtksurfacereader -ifile' + ' ' + whiskerpath + ' -ofile' + ' ' + vobpath
# impype = pypes.PypeRun(imin)

clfinder = 'vmtkcenterlines -ifile' + ' ' + vobpath + '-ofile ' + cldatpath
clpype = pypes.PypeRun(clfinder)

# cl_out = 'vmtksurfacewriter -ifile ' + ' ' + clpath + '-ofile ' + ' ' + cl_opath
# exppype = pypes.PypeRun(cl_out)

# numpyconverter = 'vmtkCenterlinesToNumpy -ifile ' + clpath #+ ' -ofile ' + clcoords
# nppype = pypes.PypeRun(numpyconverter)

#bad bad doesn't work
# centerlineReader = vmtkscripts.vmtkSurfaceReader()
# centerlineReader.InputFileName = cldatpath
# centerlineReader.Execute()
# clNumpyAdaptor = vmtkscripts.vmtkCenterlinesToNumpy()
# clNumpyAdaptor.Centerlines = centerlineReader.Surface
# clNumpyAdaptor.Execute()
# numpyCenterlines = clNumpyAdaptor.ArrayDict


Automatic piping vmtkcenterlines
Parsing options vmtkcenterlines
    SurfaceInputFileName = C:/Users/corma/Desktop/HLAB_seals/whiskers_temp/cur/d_Har20190702_short_whiskers05/whiskers/smoothed/W11_HR_442_t_0001_150.vtp
    CenterlinesOutputFileName = C:/Users/corma/Desktop/HLAB_seals/whiskers_temp/cur/d_Har20190702_short_whiskers05/whiskers/smoothed/W11_HR_442_t_0001_150_cl.dat
Explicit piping vmtkcenterlines
Input vmtkcenterlines members:
    Id = 0
    Disabled = 0
    Surface = None
    SurfaceInputFileName = C:/Users/corma/Desktop/HLAB_seals/whiskers_temp/cur/d_Har20190702_short_whiskers05/whiskers/smoothed/W11_HR_442_t_0001_150.vtp
    SeedSelectorName = pickpoint
    SourceIds = []
    TargetIds = []
    SourcePoints = []
    TargetPoints = []
    AppendEndPoints = 0
    CheckNonManifold = 0
    FlipNormals = 0
    CapDisplacement = 0.0
    DelaunayTolerance = 0.001
    RadiusArrayName = MaximumInscribedSphereRadius
    AppendEndPoints = 0
    Resampling = 0
    ResamplingStepLe

In [13]:
#convert .dat to csv

import csv

fullpath = r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\cur\d_Har20190702_short_whiskers05\whiskers\smoothed\W11_HR_442_t_0001_150.stl"

(proot, file) = os.path.split(fullpath)
whiskername = os.path.splitext(file)[0]

dfile = goodpath(proot + "/" + whiskername + "_cl.dat")
cfile = goodpath(proot + "/" + whiskername + "_cl.csv")

with open(dfile) as dat_file, open(cfile, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)
     

In [ ]:
# plots centerline in 3D with equal axes

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import os
import pathlib

%matplotlib notebook

def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])


wpath = r"C:\Users\corma\Desktop\cltest.txt"
p1 = pathlib.PureWindowsPath(wpath)
clpath = p1.as_posix()

clfine = open(wpath, "r")
endcondit = False

xcoords = []
ycoords = []
zcoords = []

while (endcondit == False): #just goes until it can't go any more
    ln = clfine.readline() #reads lines consecutively

    if (len(ln)==0): #since length is only 0 at the end
        endcondit = True
        break #goodbye
    coords = ln.split()
    xcoords.append(float(coords[0]))
    ycoords.append(float(coords[1]))
    zcoords.append(float(coords[2]))

    
fig = plt.figure()
ax = fig.gca(projection='3d')
# ax.set_aspect('equal')

ax.scatter(xcoords, ycoords, zcoords)

set_axes_equal(ax)

plt.show()

## Automatic smoothing for each isolated whisker 

Scroll down a bit and there's a tester cell for different types of smoothing

In [ ]:
# not finished yet, don't use

import os
import pathlib
import time
import gc

gc.enable()
rootdir = r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\cur" #give it a whisker folder

for root, dirs, files in os.walk(rootdir):
    for dir in dirs:
        if dir == 'whiskers':
            folderpath = os.path.join(root,dir)
            newdirs = os.listdir(folderpath)
            if len(newdirs) != 0:
                for dir in newdirs:
                    if dir.endswith('.stl') == True and dir.endswith('_cl.stl') == False:
                        basename = os.path.splitext(dir)[0]
                        inpath = goodpath(root + "/whiskers/" + dir)
#                         print(inpath)
#                         print(os.path.split(inpath))
#                         print(os.path.splitext(os.path.split(inpath)[1])[0], '\n')
                        importargcreator(inpath)
                        subdivider(inpath)
                        choosesmoother(path)
                        time.sleep(5)
                        gc.collect()
                        
#                         ins = importargcreator(inpath)
                        
                   
#             for file in files:
#                 print(root, '\n')
#                 print(dir, '\n')
#                 print(file, '\n', '\n')
                
#                 mast_path = goodpath(root)
#                 dicom_path = goodpath(os.path.join(dir, firstdcm))
                
                
#                 (pp, whiskerstl) = os.path.split(whiskerpath)
# basename = os.path.splitext(whiskerstl)[0]



In [ ]:
def importargcreator(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
    curfile = proot + "/smoothed/" + whiskername + '_isol_surf.vtp '
    arg_8 = 'vmtksurfacereader -ifile ' + path + ' -ofile ' + curfile
    try:
        EightPype = pypes.PypeRun(arg_8)
    except Exception:
        pass
    


In [ ]:
def subdivider(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0]
    try:
        nameparts = whiskername.split('_')
        mode = nameparts[1] 
    except Exception:
        mode = 'LR'
        
        
    dir_6 = proot + "/smoothed/" + whiskername + '_isol_surf.vtp ' # smoothed surface
    dir_7 = proot + "/smoothed/" + whiskername + '_sb_surf.vtp ' # surface subdivision
    
    if mode == 'HR': #high resolution, 34um
        arg_5 = 'vmtksurfacesubdivision -subdivisions 2 -method butterfly -ifile ' + dir_6 + '-ofile ' + dir_7 # subdivides, butterfly keeps vertices but makes surface more continuous
    elif mode == 'LR':
        arg_5 = 'vmtksurfacesubdivision -subdivisions 3 -method butterfly -ifile ' + dir_6 + '-ofile ' + dir_7 
        

    try:
        sbpype = pypes.PypeRun(arg_5)
    except Exception:
        pass
    


In [ ]:
def linearsubdivider(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0]
    try:
        nameparts = whiskername.split('_')
        mode = nameparts[1] 
    except Exception:
        mode = 'LR'
        
        
    dir_6 = proot + "/smoothed/" + whiskername + '_isol_surf.vtp ' # smoothed surface
    dir_7 = proot + "/smoothed/" + whiskername + '_linsb_surf.vtp ' # surface subdivision
    
    if mode == 'HR': #high resolution, 34um
        arg_5 = 'vmtksurfacesubdivision -subdivisions 2 -method linear -ifile ' + dir_6 + '-ofile ' + dir_7 # subdivides, butterfly keeps vertices but makes surface more continuous
    elif mode == 'LR':
        arg_5 = 'vmtksurfacesubdivision -subdivisions 3 -method linear -ifile ' + dir_6 + '-ofile ' + dir_7 
        

    try:
        sbpype = pypes.PypeRun(arg_5)
    except Exception:
        pass

In [ ]:
def taubin01_150_linsub(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp ' 
    curfile = proot + "/smoothed/" + whiskername+ '_linsb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_lin_t_01_150.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_lin_t_01_150.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.1 -iterations 150 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def taubin001_150_linsub(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp ' 
    curfile = proot + "/smoothed/" + whiskername+ '_linsb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_lin_t_001_150.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_lin_t_001_150.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.01 -iterations 150 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def taubin02_150(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp ' 
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_t_02_250.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_t_02_250.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.2 -iterations 250 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def taubin01_150(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp ' 
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_t_01_150.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_t_01_150.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.1 -iterations 150 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def taubin005_150(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp '
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_t_005_150.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_t_005_150.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.05 -iterations 150 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def taubin002_150(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp '
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_t_002_150.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_t_002_150.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.02 -iterations 150 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def taubin001_150(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp '
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_t_001_150.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_t_001_150.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.01 -iterations 150 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def taubin0005_150(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp ' 
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_t_0005_150.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_t_0005_150.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.005 -iterations 150 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def taubin0002_150(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp ' 
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_t_0002_150.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_t_0002_150.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.002 -iterations 150 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def taubin0001_150(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
#     curfile = proot + "/smoothed/" + whiskername+ '_isol_surf.vtp ' 
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_t_0001_150.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_t_0001_150.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -passband 0.001 -iterations 150 -ofile ' + outfile ###taubin smoothing, originally arg_4
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TP = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        SP = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
def choosesmoother(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0]
    
    try:
        nameparts = whiskername.split('_')
        mode = nameparts[1]
        diam = nameparts[2]
        
        if mode == "LR":
            if diam < 700:
                taubin0001(path)
            elif diam > 1100:
                taubin0005(path)
            else:
                taubin0002(path)
                
        elif mode == "HR":
            if diam < 700:
                taubin0001(path)
            elif diam > 900:
                taubin0005(path)
            else:
                taubin0002(path)
            
    except Exception:
        pass
    

In [ ]:
def laplace_03(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
    
    curfile = proot + "/smoothed/" + whiskername + + '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_laplace50.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_lapace50.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -method "laplace" -relaxation 0.30 -iterations 50 -ofile ' + outfile ###laplace smoothing method, relaxation .07 is good
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TPype = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        Spipe = pypes.PypeRun(arg_7)
    except Exception:
        pass
    
    
#         dir_3 = mast_dir + 'mc_surface.vtp '
#     dir_10 = mast_dir + 'mc_surf_out.stl '
#     arg_7 = 'vmtksurfacewriter -ifile ' + dir_3 + '-ofile ' + dir_10

In [ ]:
def laplace_15(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
    
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_laplace15.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_lapace15.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -method "laplace" -relaxation 0.15 -iterations 50 -ofile ' + outfile ###laplace smoothing method, relaxation .07 is good
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TPype = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        Spipe = pypes.PypeRun(arg_7)
    except Exception:
        pass
    
    
#         dir_3 = mast_dir + 'mc_surface.vtp '
#     dir_10 = mast_dir + 'mc_surf_out.stl '
#     arg_7 = 'vmtksurfacewriter -ifile ' + dir_3 + '-ofile ' + dir_10

In [ ]:
#good for short whiskers

def laplace_09_100(path):
    (proot, file) = os.path.split(path)
    whiskername = os.path.splitext(file)[0] #strips .stl
    
    curfile = proot + "/smoothed/" + whiskername+ '_sb_surf.vtp '
    outfile = proot + "/smoothed/" + whiskername + '_laplace09_100.vtp '
    outstl = proot + "/smoothed/" + whiskername + '_lapace09_100.stl '
    
    arg_3 = 'vmtksurfacesmoothing -ifile ' + curfile + ' -method "laplace" -relaxation 0.09 -iterations 100 -ofile ' + outfile ###laplace smoothing method, relaxation .07 is good
    arg_7 = 'vmtksurfacewriter -ifile ' + outfile + ' -ofile ' + outstl
    
    try:
        TPype = pypes.PypeRun(arg_3)
    except Exception:
        pass
    
    try:
        Spipe = pypes.PypeRun(arg_7)
    except Exception:
        pass

In [ ]:
# for testing different smoothing parameters, give it some whiskers to work with
import os
import pathlib
import time
import gc

gc.enable()

# w5 = goodpath(r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\d_Har20190708_harbor3_set3_fullLength\whiskers\W5.stl")
# w10 = goodpath(r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\d_Har20190708_harbor3_set3_fullLength\whiskers\W10.stl")
# w16 = goodpath(r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\d_Har20190708_harbor3_set3_fullLength\whiskers\W16.stl")
# sw1 = goodpath(r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\higherres\d_Har20190708_harbor5_set2\whiskers\W1.stl")
# sw7 = goodpath(r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\higherres\d_Har20190708_harbor5_set2\whiskers\W7.stl")
# sw10 = goodpath(r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\higherres\d_Har20190708_harbor5_set2\whiskers\W10.stl")
# sw15 = goodpath(r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\higherres\d_Har20190708_harbor5_set2\whiskers\W15.stl")
# sw20 = goodpath(r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\higherres\d_Har20190708_harbor5_set2\whiskers\W20.stl")

# whiskerlist = [w5, w10, w15, sw1, sw7, sw10, sw15, sw20]

# for inpath in whiskerlist:
#     importargcreator(inpath)
#     laplace_03(inpath)
#     taubin005_100(inpath)
#     laplace_09_100(inpath)
#     time.sleep(1)
#     gc.collect()

testwhisker = goodpath(r"C:\Users\corma\Desktop\HLAB_seals\whiskers_temp\d_Har20190708_harbor4_set2_fullLength\whiskers\mc_surf_out.stl")



# give it a path of an isolated whisker stl as "inpath", run importargcreator and subdivider, and then whichever smoothing methods you want

inpath = testwhisker
importargcreator(inpath)
subdivider(inpath)

taubin02_150(inpath)
taubin01_150(inpath)

taubin005_150(inpath)

In [ ]:
def autosmoother(path, length):
    mast_dir = ' '+path+ '/' #give folder with stripped file extension
    dir_3 = mast_dir + 'mc_surface.vtp '
    dir_6 = mast_dir + 'mc_surf_smoothed.vtp '
    if length == 'long':
        arg_3 = 'vmtksurfacesmoothing -ifile' + dir_3 + '-method "laplace" -relaxation 0.3 -iterations 50 -ofile' + dir_6 ###laplace smoothing method, relaxation .07 is good
    elif length == 'medium':
        ...
    elif length == 'short':
        ...
    

### Other assorted stuff... don't want to delete

In [ ]:
mySurface = myPype.GetScriptObject('vmtkmarchingcubes','0').Surface

mySmoother = vmtkscripts.vmtkSurfaceSmoothing()
mySmoother.Surface = mySurface
mySmoother.PassBand = 0.1
mySmoother.NumberOfIterations = 30
mySmoother.Execute()

In [ ]:
myWriter = vmtkscripts.vmtkSurfaceWriter()
myWriter.Surface = mySmoother.Surface
myWriter.OutputFileName = mast_dir + 'mysurface.vtp'
myWriter.Execute()

In [ ]:
customscript = 'customScript'

class customScript(pypes.pypeScript):
    def __init__(self):
        pypes.pypeScript.__init__(self)
        self.Image = None
        self.Surface = None
        self.Levels=[]
        self.SetScriptName('customScript')
        self.SetScriptDoc('here goes a description')
        self.SetInputMembers([
            ['Image','i','vtkImageData',1,'','the input image','vmtkimagereader'],
            ['Levels','levels','float',-1,'','graylevels to generate the isosurface at'],
            ])
        self.SetOutputMembers([
            ['Surface','o','vtkPolyData',1,'','the output surface','vmtksurfacewriter']
            ])
    def Execute(self): 
        if self.Image == None:
            self.PrintError('Error: No Image.')
        if self.Levels == []:
            self.PrintError('Error: No Levels')
        self.marchingCubes = vmtkscripts.vmtkMarchingCubes()
        self.marchingCubes.Image = self.Image
        self.marchingCubes.Connectivity = 1
        self.vmtkRenderer = vmtkscripts.vmtkRenderer()
        self.vmtkRenderer.Initialize()
        self.SurfaceViewer = vmtkscripts.vmtkSurfaceViewer()
        self.SurfaceViewer.vmtkRenderer = self.vmtkRenderer
        for level in self.Levels:
            self.marchingCubes.Level = level
            self.marchingCubes.Execute()
            self.Surface = self.marchingCubes.Surface
            self.SurfaceViewer.Surface = self.Surface
            self.SurfaceViewer.BuildView()     
    pass

if __name__=='__main__':
    main = pypes.pypeMain()
    main.Arguments = sys.argv
    main.Execute()

In [ ]:
     vmtkcenterlines -seedselector openprofiles -ifile tubeExtended.stl -ofile centerlines.vtp  --pipe vmtksurfacewriter -ifile centerlines.vtp -ofile centerlines.vtk